In [37]:
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
np.random.seed(42)

# we generate a three-dimensional random vector by sampling
# each entry from a standard normal distribution
v = np.random.normal(0, 1, 3)

# purity of the target state
purity = 0.66

# create a random Bloch vector with the specified purity
bloch_v = Variable(
    torch.tensor(np.sqrt(2 * purity - 1) * v / np.sqrt(np.sum(v ** 2))),
    requires_grad=False
)
print(bloch_v)
# array of Pauli matrices (will be useful later)
Paulis = Variable(torch.zeros([4, 2, 2], dtype=torch.complex128), requires_grad=False)
Paulis[0] = torch.tensor([[0, 1], [1, 0]])
Paulis[1] = torch.tensor([[0, -1j], [1j, 0]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])

tensor([ 0.3394, -0.0945,  0.4426], dtype=torch.float64)


In [38]:
# number of qubits in the circuit
nr_qubits = 4
# number of layers in the circuit
nr_layers = 2

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, (nr_qubits, nr_layers, 3))
params = Variable(torch.tensor(params), requires_grad=True)

# a layer of the circuit ansatz
def layer(params, j):
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 3])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[2, 3])

In [39]:
dev = qml.device("default.qubit", wires=4)

In [40]:
@qml.qnode(dev, interface="torch")
def circuit(params, A):

    # repeatedly apply each layer in the circuit
    for j in range(nr_layers):
        layer(params, j)

    # returns the expectation of the input matrix A on the first qubit
    return qml.expval(qml.Hermitian(A, wires=0))

In [41]:
# cost function
def cost_fn(params):
    cost = 0
    for k in range(3):
        cost += torch.abs(circuit(params, Paulis[k]) - bloch_v[k])
        #print("Bloch_v is {:.4f}".format(bloch_v[k]))

    return cost


# set up the optimizer
opt = torch.optim.Adam([params], lr=0.1)

# number of steps in the optimization routine
steps = 200

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((nr_qubits, nr_layers, 3))

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = cost_fn(params)
    loss.backward()
    opt.step()

    # keeps track of best parameters
    if loss < best_cost:
        best_cost = loss
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, loss))

# calculate the Bloch vector of the output state
output_bloch_v = np.zeros(3)
for l in range(3):
    output_bloch_v[l] = circuit(best_params, Paulis[l])

# print results
print("Target Bloch vector = ", bloch_v.numpy())
print("Output Bloch vector = ", output_bloch_v)

Cost after 0 steps is 0.9276
Cost after 10 steps is 0.1652
Cost after 20 steps is 0.1531
Cost after 30 steps is 0.1069
Cost after 40 steps is 0.0423
Cost after 50 steps is 0.0468
Cost after 60 steps is 0.0628
Cost after 70 steps is 0.0333
Cost after 80 steps is 0.0222
Cost after 90 steps is 0.0186
Cost after 100 steps is 0.0406
Cost after 110 steps is 0.0201
Cost after 120 steps is 0.0434
Cost after 130 steps is 0.0392
Cost after 140 steps is 0.0831
Cost after 150 steps is 0.0624
Cost after 160 steps is 0.0549
Cost after 170 steps is 0.0831
Cost after 180 steps is 0.0712
Cost after 190 steps is 0.0598
Cost after 200 steps is 0.0412
Target Bloch vector =  [ 0.33941241 -0.09447812  0.44257553]
Output Bloch vector =  [ 0.38340777 -0.11759117  0.42930696]
